In [ ]:
import os
import sys
import warnings
sys.path.append(os.path.dirname(os.getcwd()))

import yaml

In [ ]:
from oligo_designer_toolsuite.pipelines import TargetProbes

In [ ]:
from oligo_designer_toolsuite.utils import (
    parse_fasta_header,
    check_fasta_format,
)

In [ ]:
config_file = "./configs/merfish_probe_designer_test.yaml"
# config_file = "./configs/padlock_probe_designer_ncbi.yaml" # NCBI config
# config_file = "./configs/padlock_probe_designer_ensemble.yaml" # Ensemble config
with open(config_file, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)
dir_output = os.path.join(os.path.dirname(os.getcwd()), config["dir_output"])

In [ ]:
from oligo_designer_toolsuite.database import CustomGenomicRegionGenerator, NcbiGenomicRegionGenerator, EnsemblGenomicRegionGenerator

# If the custom config file is selected
if config["source"] == "custom":
    region_generator_custom = CustomGenomicRegionGenerator(
        annotation_file=config["annotation_file"], 
        sequence_file=config["sequence_file"], 
        files_source=config["files_source"], 
        species=config["species"], 
        annotation_release=config["annotation_release"], 
        genome_assembly=config["genome_assembly"],
        dir_output=dir_output
    )
# If the Ncbi config file is selected
elif config["source"] == "ncbi":
    region_generator = NcbiGenomicRegionGenerator(
        taxon=config["taxon"],
        species=config["species"], 
        annotation_release=config["annotation_release"], 
        dir_output=dir_output
    )
# If the Ensembl config file is generated
elif config["source"] == "ensembl":
    region_generator = EnsemblGenomicRegionGenerator(
        species=config["species"], 
        annotation_release=config["annotation_release"], 
        dir_output=dir_output
    )

file_transcriptome = region_generator_custom.generate_transcript_reduced_representation(include_exon_junctions=True, exon_junction_size=2*config["oligo_length_max"])

In [ ]:
target_probe_class = TargetProbes(config,
            dir_output,
            file_transcriptome,
            region_generator_custom)
target_probes, file_target_probes= target_probe_class.create_target()